# 3.1 Parabolic model problem
In this unit we want to turn to instationary problems. And we will start with a very basic setup: implicit Euler time stepping for the convection diffusion equation. After the main part of this tutorial we have supplementary material to extend the case in several regards.



We are solving the unsteady heat equation 

$$\text{find } u:[0,T] \to H_{0,D}^1 \quad \int_{\Omega} \partial_t u v + \int_{\Omega} \nabla u \nabla v + b \cdot \nabla u v = \int f v  \quad \forall v \in H_{0,D}^1, \quad u(t=0) = u_0$$

with a suitable advective field $b$ (the wind).

In [1]:
#imports
from ngsolve import *
from netgen.geom2d import SplineGeometry
from netgen.geom2d import unit_square
import netgen.gui
%gui tk
import numpy as np
import matplotlib.pyplot as plt

optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2305
Using Lapack
Including sparse direct solver UMFPACK
Running parallel using 8 thread(s)


* Geometry: $(-1,1)^2$
* Dirichlet boundaries everywhere
* Mesh

In [2]:
from netgen.occ import *
shape = Rectangle(2,2).Face().Move((-1,-1,0))
shape.edges.Min(X).name="left"
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(Y).name="top"
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.25))

 Face 1 / 1 (parameter space projection)


In [3]:
fes = H1(mesh, order=3, dirichlet="bottom|right|left|top")
u,v = fes.TnT()
time = 0.0
dt = 0.001

We define the field $b$ (the wind) as 

$$b(x,y) := (2y(1-x^2),-2x(1-y^2)).$$

In [4]:
b = CoefficientFunction((2*y*(1-x*x),-2*x*(1-y*y)))
Draw(b,mesh,"wind", vectors={"grid_size": 32}, order=3)

* bilinear forms for 
 * convection-diffusion stiffness and 
 * mass matrix separately.
* non-symmetric memory layout for the mass matrix so that a and m have the same sparsity pattern.

In [5]:
a = BilinearForm(fes, symmetric=False)
a += 0.01*grad(u)*grad(v)*dx + b*grad(u)*v*dx
a.Assemble()

m = BilinearForm(fes, symmetric=False)
m += u*v*dx
m.Assemble()

## Implicit Euler method
$$
  \underbrace{(M + \Delta t A)}_{M^\ast} u^{n+1} = M u^n + \Delta tf^{n+1}
$$

or in an incremental form:

$$
  M^\ast (u^{n+1} - u^n) = - \Delta t A u^n + \Delta tf^{n+1}.
$$

* Incremental form: $u^{n+1} - u^n$ has homogeneous boundary conditions (unless boundary conditions are time-dependent).
* For the time stepping method: set up linear combinations of matrices.
* (Only) if the sparsity pattern of the matrices agree we can copy the pattern and sum up the entries.

First, we create a matrix of same format as m.mat and compare number of non-zero entries and sparsity pattern:

In [6]:
mstar = m.mat.CreateMatrix()
print(f"m.mat.nze = {m.mat.nze}, a.mat.nze={a.mat.nze}, mstar.nze={mstar.nze}")

m.mat.nze = 11257, a.mat.nze=11257, mstar.nze=11257


To access the entries we use the vector of nonzero-entries:

In [7]:
print(f"mstar.nze={mstar.nze}, len(mstar.AsVector())={len(mstar.AsVector())}")

mstar.nze=11257, len(mstar.AsVector())=11257


In [8]:
print(mstar.AsVector())

 0.00520833
 0.00260417
 0.00260417
 -0.000520833
 8.68056e-05
 -0.000520833
 8.68056e-05
 -0.000260417
 -1.32137e-19
 0.000173611
 0.00823929
 0.00207589
 0.00204376
 0.00411964
 -0.000415177
 6.91962e-05
 -0.000408752
 6.81253e-05
 -0.000823929
 0.000137321
 -0.000207589
 1.05032e-19
 -0.000204376
 -1.05032e-19
 0.000138392
 0.000136251
 0.00520833
 0.00260417
 0.00260417
 -0.000520833
 8.68056e-05
 -0.000520833
 8.68056e-05
 -0.000260417
 1.38913e-19
 0.000173611
 0.00827496
 0.00209025
 0.00204723
 0.00413748
 -0.000418049
 6.96749e-05
 -0.000409446
 6.82411e-05
 -0.000827496
 0.000137916
 -0.000209025
 1.01644e-19
 -0.000204723
 -1.05032e-19
 0.00013935
 0.000136482
 0.00260417
 0.0195488
 0.00333868
 0.00643574
 0.00717025
 -0.000520833
 -8.68056e-05
 -0.000260417
 -1.42302e-19
 -0.000667736
 0.000111289
 -0.00128715
 0.000214525
 -0.00143405
 0.000239008
 -0.000333868
 -1.69407e-19
 -0.000383157
 1.96512e-19
 0.000173611
 0.000222579
 0.000255438
 0.00333868
 0.0199715
 0.002826

Using the vector we can build the linear combination of the a and the m matrix:

In [9]:
mstar.AsVector().data = m.mat.AsVector() + dt * a.mat.AsVector()
# corresponds to M* = M + dt * A
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())

We set the r.h.s. $f = exp(-6 ((x+\frac12)^2+y^2)) - exp(-6 ((x-\frac12)^2+y^2))$

In [10]:
f = LinearForm(fes)
gaussp = exp(-6*((x+0.5)*(x+0.5)+y*y))-exp(-6*((x-0.5)*(x-0.5)+y*y))
Draw(gaussp,mesh,"f", deformation=True)
f += gaussp*v*dx
f.Assemble()

and the initial data: $u_0 = (1-y^2)x$

In [11]:
gfu = GridFunction(fes)
gfu.Set((1-y*y)*x) # note that boundary conditions remain
scene = Draw(gfu,mesh,"u")

In [12]:
scene

we define a simple time loop including some visualization sampling:

In [ ]:
def TimeStepping(invmstar, initial_cond = None, t0 = 0, tend = 20, nsamples = 10):
    if initial_cond:
        gfu.Set(initial_cond)
    cnt = 0; time = t0
    sample_int = int(floor(tend / dt / nsamples)+1)
    gfut = GridFunction(gfu.space,multidim=0)
    gfut.AddMultiDimComponent(gfu.vec)
    while time < tend - 0.5 * dt:
        res = dt * f.vec - dt * a.mat * gfu.vec
        gfu.vec.data += invmstar * res
        print("\r",time,end="")
        # scene.Redraw()
        Redraw()
        if cnt % sample_int == 0:
            gfut.AddMultiDimComponent(gfu.vec)
        cnt += 1; time = cnt * dt
    return gfut

In [20]:
%%time
gfut = TimeStepping(invmstar, (1-y*y)*x)

 19.999CPU times: user 9.49 s, sys: 991 ms, total: 10.5 s
Wall time: 9.5 s


In [15]:
Draw(gfut, mesh, interpolate_multidim=True, animate=True)

TypeError: Draw(): incompatible function arguments. The following argument types are supported:
    1. (cf: ngsolve.fem.CoefficientFunction, mesh: ngsolve.comp.Mesh, name: str, sd: int = 2, autoscale: bool = True, min: float = 0.0, max: float = 1.0, draw_vol: bool = True, draw_surf: bool = True, reset: bool = False, title: str = '', number_format: str = '%.3e', unit: str = '', **kwargs) -> None
    2. (gf: ngsolve.comp.GridFunction, sd: int = 2, autoscale: bool = True, min: float = 0.0, max: float = 1.0, **kwargs) -> None
    3. (mesh: ngsolve.comp.Mesh, **kwargs) -> None
    4. (arg0: object) -> None

Invoked with: <ngsolve.comp.GridFunction object at 0x7f4678190770>, <ngsolve.comp.Mesh object at 0x7f45c279d490>; kwargs: interpolate_multidim=True, animate=True

## Supplementary material:

The simple time stepping can be extended in several directions:

1. Instead of direct one could use iterative solvers
2. R.h.s. data time-dependent
3. boundary data time-dependent
4. Replace implicit Euler time stepping with (Singly diagonal) Runge-Kutta methods



## Supplementary 1: Using iterative solvers

* For a factorization of $M^\ast$ (and ${M^\ast}^{-1}$) we required a sparse matrix $M^\ast$ 
* To store $M^\ast$ **as a sparse matrix** requires new storage (and same memory layout of $A$ and $M$)
* For iterative solvers we only require the matrix (and preconditioner) applications and it **suffices to have $M^\ast$ available as a linear operator**
* `mstar_alt = m.mat + dt * a.mat` has no storage but defines the operator action: sum of two matrix-vector multiplications

iterative solver version (with Jacobi preconditining):

In [ ]:
mstar_alt = m.mat + dt * a.mat
premstar_alt = m.mat.CreateSmoother() + dt * a.mat.CreateSmoother()
print(premstar_alt)

Now, replace the action of the inverse Matrix from the previous script with a solution of a conjugate gradient method:

In [ ]:
from ngsolve.krylovspace import CGSolver
invmstar_alt = CGSolver(mstar_alt, premstar_alt, tol=1e-8, printrates="\r", maxiter=200)

Now, we can repeat the time stepping. As in 2D direct solvers are quite efficient, this simple preconditioned solver can hardly compete. We hence only do a few time steps:

In [16]:
%%time
gfut_a1 = TimeStepping(invmstar_alt, (1-y*y)*x, tend=25*dt, nsamples=5)

NameError: name 'invmstar_alt' is not defined

In [ ]:
Draw(gfut_a1, mesh, interpolate_multidim=True, animate=True)

## Supplementary 2: time-dependent r.h.s. data <a class="anchor" id="TDRHS"></a>

Next: time-dependent r.h.s. data $f=f(t)$:

* Use `Parameter` t representing the time. 
* A `Parameter` is a constant `CoefficientFunction` the value of which can be changed with the `Set`-function.

In [ ]:
t = Parameter(0.0)

An example of a time-dependent coefficient that we want to use as r.h.s. in the following is

In [ ]:
omega=1
gausspt = exp(-6*((x+sin(omega*t))*(x+sin(omega*t))+y*y))-exp(-6*((x-sin(omega*t))*(x-sin(omega*t))+y*y))
gff = GridFunction(L2(mesh,order=gfu.space.globalorder+1))
gfft = GridFunction(gff.space,multidim=0)
time = 0.0
for i in range(7):
    t.Set(3*i/6)
    gff.Set(gausspt)
    gfft.AddMultiDimComponent(gff.vec)
Draw(gfft, mesh, interpolate_multidim=True, animate=True, autoscale=False, min=-1, max=1)

Accordingly, we define a different linear form which then has to be assembled in every time step.

In [ ]:
ft = LinearForm(fes)
ft += gausspt*v*dx

In [ ]:
def TimeStepping_app2(invmstar, initial_cond = None, t0 = 0, tend = 2, 
                      nsamples = 10):
    if initial_cond:
        gfu.Set(initial_cond)
    cnt = 0; time = t0
    sample_int = int(floor(tend / dt / nsamples)+1)
    gfut = GridFunction(gfu.space,multidim=0)
    gfut.AddMultiDimComponent(gfu.vec)
    while time < tend - 0.5 * dt:
        t.Set(time)
        ft.Assemble()
        res = dt * ft.vec - dt * a.mat * gfu.vec
        gfu.vec.data += invmstar * res
        print("\r",time,end="")
        if cnt % sample_int == 0:
            gfut.AddMultiDimComponent(gfu.vec)
        cnt += 1; time = cnt * dt
    return gfut

In [ ]:
%%time
gfut_a2 = TimeStepping_app2(invmstar, initial_cond=CF(0),tend=2)
Draw(gfut_a2, mesh, interpolate_multidim=True, animate=True, 
     min=-0.75,max=0.75,autoscale=False)

## Supplementary 3: Time dependent boundary conditions <a class="anchor" id="TDBND"></a>


* $u|_{\partial \Omega} = u_D(t)$, $f=0$
* implicit Euler time stepping method, non-incremental form:

  $$
    M^\ast u^{n+1} = (M + \Delta t A) u^{n+1} = M u^n
  $$  
  
* Homogenize w.r.t. to boundary conditions, i.e. we split 

  $$ u^{n+1} = u^{n+1}_0 + u^{n+1}_D $$ 
  
  where $u^{n+1}_D$ is a (discrete) function with correct boundary condition:  
  
$$
  {M^\ast} u^{n+1}_0 = M u^n - {M^\ast} u^{n+1}_D
$$


In [ ]:
uD = CoefficientFunction( [(1-x*x)*IfPos(sin(0.3*pi*t),sin(0.3*pi*t),0),0,0,0])
time = 0.0
t.Set(0.0)
gfu.Set(uD,BND)

In [ ]:
def TimeStepping_app3(invmstar, initial_cond = None, t0 = 0, tend = 2, 
                      nsamples = 10):
    if initial_cond:
        gfu.Set(initial_cond)
    cnt = 0; time = t0
    sample_int = int(floor(tend / dt / nsamples)+1)
    gfuD = GridFunction(gfu.space)
    gfut = GridFunction(gfu.space,multidim=0)
    gfut.AddMultiDimComponent(gfu.vec)
    while time < tend - 0.5 * dt:
        t.Set(time)
        gfuD.Set(uD,BND)
        res = m.mat * gfu.vec - mstar *gfuD.vec
        gfu.vec.data = gfuD.vec + invmstar * res
        print("\r",time,end="")
        if cnt % sample_int == 0:
            gfut.AddMultiDimComponent(gfu.vec)
        cnt += 1; time = cnt * dt
    return gfut

In [ ]:
%%time
gfut_a3 = TimeStepping_app3(invmstar, initial_cond=CF(0),tend=10)
Draw(gfut_a3, mesh, interpolate_multidim=True, animate=True, 
     #settings = {"subdivision" : 10}, 
     deformation = True, min = 0, max = 0.2, autoscale = False)

## Supplementary 4: Singly diagonally implicit Runge-Kutta methods <a class="anchor" id="RK"></a>


Let us consider more sophisticated time integration methods, SDIRK methods. To simplify presentation we set $f=0$.

SDIRK methods for the semi-discrete problem $\frac{d}{dt} u = M^{-1} F(u) = -M^{-1} \cdot A u$ are of the form:

$$
  u^{n+1} = u^n + \Delta t M^{-1} \sum_{i=0}^{s-1} b_i k_i
$$

with

$$
k_i = - A u_i \quad \text{ where $u_i$ is the solution to } \quad 
(M + a_{ii} \Delta t A) u_i = M u^n + \Delta t \sum_{j=0}^{i-1} a_{ij} k_j, \quad i=0,..,s-1.
$$

The coefficients a,b and c are stored in the butcher tableau:

$$
\begin{array}{c|cccc}
c_0 & a_{00} & 0 & \ddots& \\
c_1 & a_{10} & a_{11} & 0 & \ddots \\
\vdots & \vdots & \vdots & \ddots & 0\\
c_{s-1} & a_{s-1,0} & a_{s-1,1} & \dots& a_{s-1,s-1} \\ \hline
 & b_{0} & b_1 & \dots& b_{s-1} \\
\end{array}
$$

For an SDIRK method we have $a^\ast = a_{ii},~i=0,..,s-1$.

Simplest example: Implicit Euler
$$
\begin{array}{c|c}
1 & 1 \\ \hline
 & 1  \\
\end{array}
$$

In [ ]:
class sdirk1: #order 1 (implicit Euler)
    stages = 1
    a = [[1]]
    b = [1]
    c = [1]
    astar = 1

Midpoint rule:

In [ ]:
class MR: #order 2 (midpoint rule)
    stages = 1
    a = [[0.5]]
    b = [1]
    c = [0.5]
    astar = 0.5

We can use for example the 2 stage SDIRK (order 3) method

$$
\begin{array}{c|cc}
p & p & 0 \\
1-p & 1-2p & p \\ \hline
 & 1/2 & 1/2 \\
\end{array}
$$

with $p = \frac{3 - \sqrt{3}}{6}$.

In [ ]:
class sdirk2: #order 3
    p = (3 - sqrt(3))/6
    stages = 2
    a = [[p, 0], 
         [1 - 2*p, p]]
    b = [1/2, 1/2]
    c = [p, 1 - p]
    astar = p
    

or the 5 stage SDIRK (order 4) method
$$
\begin{array}{c|cccc}
1/4 & 1/4  \\
3/4 & 1/2 & 1/4 & \\
11/20 & 17/50 & -1/25 & 1/4 \\
1/2 & 371/1360 & -137/2720 & 15/544 & 1/4 \\
1 & 25/4 & -49/48 & 125/16 & -85/12 & 1/4 \\ \hline
 & 25/4 & -49/48 & 125/16 & -85/12 & 1/4 \\
\end{array}
$$

In [ ]:
class sdirk5: #order 4
    stages = 5
    a = [[1/4, 0, 0, 0, 0], 
         [1/2, 1/4, 0, 0, 0], 
         [17/50,-1/25, 1/4, 0, 0],
         [371/1360, -137/2720, 15/544, 1/4,0],
         [25/24, -49/48, 125/16, -85/12, 1/4]]
    b = [25/24, -49/48, 125/16, -85/12, 1/4]
    c = [1/4, 3/4, 11/20, 1/2, 1]
    astar = 1/4    

We have to update the $M^\ast$ matrix due to the different diagonal in the butcher tableau.
Further, we will also need the inverse mass matrix to compute the slope states $k_i$. 

In the next function, we put setup of the matrices, their inverses and the time stepping in one (larger) loop:

In [ ]:
def TimeStepping_app4(m, a, butchertab, dt,
                      initial_cond = None, t0 = 0, tend = 2, 
                      nsamples = 10):
    mstar.AsVector().data = m.mat.AsVector() + butchertab.astar * dt * a.mat.AsVector()
    invmstar = mstar.Inverse(freedofs=fes.FreeDofs())
    invmass = m.mat.Inverse(freedofs=fes.FreeDofs())    
    
    rhsi, Mu0, ui = gfu.vec.CreateVector(),gfu.vec.CreateVector(),gfu.vec.CreateVector()   
    k = [gfu.vec.CreateVector() for i in range(butchertab.stages)]    

    if initial_cond:
        gfu.Set(initial_cond)
    cnt = 0; time = t0
    sample_int = int(floor(tend / dt / nsamples)+1)
    gfuD = GridFunction(gfu.space)
    gfut = GridFunction(gfu.space,multidim=0)
    gfut.AddMultiDimComponent(gfu.vec)
    while time < tend - 0.5 * dt:
        Mu0.data = m.mat * gfu.vec
        for i in range(butchertab.stages):
            rhsi.data = Mu0
            for j in range(0,i):
                rhsi.data += dt * butchertab.a[i][j] * k[j]
            t.Set(time+butchertab.c[i]*dt)
            gfu.Set(uD,BND)
            ui.data = gfu.vec; rhsi.data -= mstar * ui
            ui.data += invmstar * rhsi
            k[i].data = - a.mat * ui
        t.Set(time+dt)
        gfu.Set(uD,BND)
        Mu0.data -= m.mat * gfu.vec
        for i in range(0,butchertab.stages):
            Mu0.data += dt * butchertab.b[i] * k[i]
        gfu.vec.data += invmass * Mu0        
        print("\r",time,end="")
        if cnt % sample_int == 0:
            gfut.AddMultiDimComponent(gfu.vec)
        cnt += 1; time = cnt * dt
    return gfut

Next, we simply apply the time stepping:

In [ ]:
gfut_a4 = TimeStepping_app4(m, a, butchertab = sdirk5(), 
                            dt = 0.25, initial_cond=CF(0),tend=10)
Draw(gfut_a4, mesh, interpolate_multidim=True, animate=True, 
     order = 3, 
     deformation = True, min = 0, max = 0.2, autoscale = False)

**Tasks**

If you want to play around and explore this tutorial unit, here are a few suggestions:

* Make a convergence study (for convergence in space, time and space and time)
* Re-introduce non-zero r.h.s. in the RK-script
* Generalize the RK script to fully implicit RK schemes (using product spaces, or block matrices, ...)



















